In [ ]:
from ipywwt import WWTWidget
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from time import sleep

In [ ]:
import ipywidgets

output = ipywidgets.Output()

@output.capture()
def on_msg(_widget, msg, buffers):
    pretty_msg = msg
    for k,v in msg.items():
        if isinstance(v, list):
            pretty_msg[k] = v[:2]
        elif isinstance(v,str) and (len(v) > 80):
            pretty_msg[k] = v[:80]
    print(f'Received: {(pretty_msg)}')

@output.capture()
def wwt_cb(wwt, updated):
    print(updated)

@output.capture()
def ready():
    print('ready')

wwt1 = WWTWidget()


# Setup the view
wwt1.background = "USNOB: US Naval Observatory B 1.0 (Synthetic, Optical)"
wwt1.foreground = "SDSS: Sloan Digital Sky Survey (Optical)"
wwt1.center_on_coordinates(SkyCoord(285.0130833333333 * u.deg,40.22075 * u.deg), instant=True)


def add_layer():
    """
    layers will only be added after the app is mounted. 
    They also seem to need a bit more time, otherwise sometimes
        it will not show up.
    """
    sleep(.1)
    OEC = '/Applications/John/github/minids/radwave/src/assets/radwave/RW_cluster_oscillation_0_updated_radec.csv'

    table = Table.read(OEC, delimiter=',', format='ascii.basic')

    layer = wwt1.layers.add_table_layer(table=table, frame='Sky',
                               lon_att='ra', lat_att='dec')
    layer.size_scale = 200
    layer.type = "gaussian"
    layer.color = "#00FF00"
    
    


# on ready is the first thing run after queued messages
wwt1.on_ready(ready)

# then callbacks added after are run in order. These Callbacks take no parameters. 
wwt1.ensure_mounted(add_layer)
# wwt1.ensure_mounted(style_layer)


wwt1.on_msg(on_msg)
wwt1.set_selection_change_callback(wwt_cb)

# wwt1.on_msg(lambda w, m, b: wwt1._on_app_message_received(m, b))
    

ipywidgets.VBox([wwt1, output])